In [4]:
!git clone -b atm-10 http://github.com/Zapy67/ATML-PA-4 

fatal: destination path 'ATML-PA-4' already exists and is not an empty directory.


In [ ]:
!git pull -b atm-10 http://github.com/Zapy67/ATML-PA-4 

From http://github.com/Zapy67/ATML-PA-4
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [5]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [6]:
!ls

'ATML - PA 4.pdf'   fed_lib	   LICENSE     task1.ipynb   task3.ipynb
 data		    fedsam.ipynb   README.md   task2.ipynb   task4.ipynb


#  FedSAM on heterogeneous domains

In [7]:
import torch
import torch.nn as nn

In [8]:
import fed_lib
import importlib
importlib.reload(fed_lib)


<module 'fed_lib' from '/kaggle/working/ATML-PA-4/fed_lib/__init__.py'>

In [6]:
!pip install fedlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.2 MB/s eta 0:

In [9]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(utils)
importlib.reload(fed_methods)
importlib.reload(fed_model)

22:28:05 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


<module 'fed_lib.fed_model' from '/kaggle/working/ATML-PA-4/fed_lib/fed_model.py'>

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
trainset, testset = utils.get_cifar10()

In [12]:
client_loaders = utils.get_heterogenous_domains(trainset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [13]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.079 0.068 0.088 0.067 0.131 0.088 0.111 0.03  0.213 0.126]
Entropy of Label Distribution H(P(Y)): 2.102870626465239
Normalized Entropy of Label Distribution: 0.9132651092302875

Label Distribution (P(Y)): [0.152 0.122 0.153 0.157 0.033 0.006 0.009 0.252 0.116]
Entropy of Label Distribution H(P(Y)): 1.8258316532923375
Normalized Entropy of Label Distribution: 0.8309717960218086

Label Distribution (P(Y)): [0.115 0.05  0.04  0.011 0.081 0.052 0.12  0.1   0.103 0.328]
Entropy of Label Distribution H(P(Y)): 1.927983508742169
Normalized Entropy of Label Distribution: 0.8373125990471938

Label Distribution (P(Y)): [0.1   0.042 0.08  0.191 0.165 0.204 0.219]
Entropy of Label Distribution H(P(Y)): 1.768351768347601
Normalized Entropy of Label Distribution: 0.9087530424804496

Label Distribution (P(Y)): [0.049 0.222 0.128 0.052 0.089 0.156 0.044 0.113 0.056 0.091]
Entropy of Label Distribution H(P(Y)): 2.073209740959855
Normalized Entropy of Label Distribution: 0.9

In [14]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [15]:
client_weights

[0.2187, 0.22118, 0.17032, 0.20554, 0.18426]

In [23]:
partition = "dirichlet"
alpha = 0.1
num_clients = 5
domains=None
fed = fed_model.Federation(num_clients, None, partition, domains, alpha, device, batch_size=64, pin_memory=True, num_workers=2)

In [24]:
client_loaders = fed.client_dataloaders
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]

In [25]:
fed.set_method(fed_methods.FedSAM(sam_rho=0, 
             num_local_steps=len(trainset)/num_clients, 
             client_aggregation_weights= client_weights))

In [37]:
import numpy as np
list(utils.SmallCNN().parameters())

[Parameter containing:
 tensor([[[[-0.1824, -0.0630,  0.0254],
           [ 0.1401, -0.0197, -0.0389],
           [ 0.0319, -0.0982, -0.0596]],
 
          [[-0.1564, -0.0116,  0.1150],
           [ 0.1410,  0.0897, -0.0470],
           [-0.0430, -0.0646, -0.0287]],
 
          [[-0.1313, -0.1540,  0.1704],
           [-0.1034,  0.1507,  0.1167],
           [-0.0448, -0.0309,  0.1785]]],
 
 
         [[[-0.1328, -0.1646,  0.0543],
           [-0.1545,  0.0200, -0.0040],
           [-0.0724,  0.0293, -0.1763]],
 
          [[-0.0928,  0.1116,  0.0440],
           [-0.1544,  0.1698,  0.1616],
           [-0.0544, -0.0817, -0.0476]],
 
          [[ 0.1456, -0.1223,  0.1681],
           [ 0.0542, -0.1107, -0.0337],
           [-0.0403, -0.1178,  0.0614]]],
 
 
         [[[-0.0321,  0.1250, -0.0090],
           [ 0.0181, -0.0425, -0.1068],
           [ 0.0123, -0.0468,  0.0771]],
 
          [[ 0.0348,  0.0894,  0.0949],
           [ 0.0702,  0.1732,  0.0368],
           [ 0.1065, -0.0481, 

In [26]:
fed.train(25, lr=1e-3, verbose=False)


--- Round 1/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 1:


FedSAM  | Test Loss: 2.3046, Test Acc: 10.00%

--- Round 2/25 ---
Training Clients
Training Client 1/5


Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 2:


FedSAM  | Test Loss: 2.3046, Test Acc: 10.00%

--- Round 3/25 ---
Training Clients
Training Client 1/5


Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 3:


FedSAM  | Test Loss: 2.3046, Test Acc: 10.00%

--- Round 4/25 ---
Training Clients
Training Client 1/5


Training Client 2/5


KeyboardInterrupt: 

In [127]:
fed.clients[0]

SmallCNN(
  (backbone): Sequential(
    (0): SmallConvBlock(in_channels=3, out_channels=32)
    (1): SmallConvBlock(in_channels=32, out_channels=64)
    (2): AvgPool2d(kernel_size=1, stride=1, padding=0)
  )
  (task_head): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)